In [ ]:
import cv2
import numpy as np
import scipy
from PIL import Image
from numpy import convolve
from numpy.fft import fft2, ifft2, fftshift, ifftshift
from numpy.linalg import norm
from scipy.sparse import csgraph
from scipy.optimize import minimize
from scipy.signal import convolve2d
from scipy.ndimage import median_filter
import matplotlib.pyplot as plt
from scipy.optimize import least_squares

# ***Grayscale images***

In [ ]:
def normalize(image, min_value=0, max_value=1):
    # Obtener los valores mínimo y máximo de la imagen
    image_min = np.min(image)
    image_max = np.max(image)

    # Normalizar la imagen en el rango especificado
    normalized_image = (image - image_min) * (max_value - min_value) / (image_max - image_min) + min_value

    return normalized_image

In [ ]:
def calcular_gradiente1D(imagen):
    """
    Calcula el gradiente de una imagen a color utilizando el método de las diferencias finitas.

    Parámetros:
        imagen (numpy.ndarray): La imagen de entrada en formato RGB. Debe ser una matriz de tamaño (alto, ancho, 3).

    Retorna:
        gradiente (numpy.ndarray): El gradiente de la imagen, con las derivadas parciales en cada canal de color.
                                   Tiene el mismo tamaño que la imagen de entrada.
    """
    alto, ancho = imagen.shape

    # Inicializar matrices para las derivadas parciales en cada canal de color
    dI_dx = np.zeros_like(imagen)
    dI_dy = np.zeros_like(imagen)

    # Calcular las derivadas parciales utilizando diferencias finitas
    dI_dx[:, 1:ancho] = imagen[:, 1:ancho] - imagen[:, :ancho-1]  # Diferencia en dirección x
    dI_dy[1:alto, :] = imagen[1:alto, :] - imagen[:alto-1, :]  # Diferencia en dirección y


    # Calcular el gradiente total combinando los gradientes de cada canal
    gradient = np.array([dI_dx, dI_dy])

    return gradient

In [ ]:
def update_u(f, q_k, lambda_k, alpha, K, K_ast, r):
    n, m = f.shape
    #, Dx, Dy = get_kernels(n,m)

    mu_q0 = lambda_k[0] + r*q_k[0]
    mu_q1 = lambda_k[1] + r*q_k[1]

    L = fft2(np.array([[0, 1, 0],[1, -4, 1],[0, 1, 0]]),s = (n,m))
    Dx = fft2(np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]]),s = (n,m))
    Dy = fft2(np.array( [[-1, -1, -1], [0, 0, 0], [1, 1, 1]]),s = (n,m))

    num = alpha*fft2(K_ast,s = (n,m))*fft2(f) - Dx*mu_q0 - Dy*mu_q1
    den = alpha*fft2(K_ast,s = (n,m))*fft2(K,s = (n,m)) - r*L

    u = np.abs(ifft2(num / den))

    return u

In [ ]:
def update_q(u_k, lambda_k, r):
    w      = np.array(np.gradient(u_k)) - lambda_k/r
    w_norm = np.sqrt(w[0]**2 + w[1]**2)

    # q      = np.zeros_like(w, dtype = np.complex64)

    q = (1 - 1/(r*w_norm))*w
    q[np.array([w_norm,w_norm]) < 1/r] = 0

    #for i in range(u_k.shape[0]):
    #    for j in range(u_k.shape[1]):
    #        norma = norm(w[:,i,j])
    #        if norma > 1/r:
    #            q[:,i,j] = (1 - 1/(r*norma))*w[:,i,j]
    #        else:
    #            q[:,i,j] = np.zeros_like(w[:,i,j], dtype = np.complex64)

    return q

In [ ]:
def q_a(x, q_l, q_l1, du, mu_k, beta, r, a):
    sum2 = np.zeros_like(x[0,:,:,0]).astype(np.complex128)
    qq   = np.zeros_like(x[0,:,:,0]).astype(np.complex128)
    pp   = np.zeros_like(x[0,:,:,0]).astype(np.complex128)
    qpq  = np.zeros_like(x[0,:,:,0]).astype(np.complex128)
    pqp  = np.zeros_like(x[0,:,:,0]).astype(np.complex128)
    ppqq = np.zeros_like(x[0,:,:,0]).astype(np.complex128)

    for i in range(3):
        for j in range(3):
            if j != i:
                sum2 += (q_l1[0,:,:,i]*q_l1[1,:,:,j] + q_l1[0,:,:,j]*q_l1[1,:,:,i])**2
                qq   += q_l[1,:,:,j]
                pp   += q_l[0,:,:,j]
                qpq  += q_l[1,:,:,j]*q_l[0,:,:,j]*q_l[1,:,:,a]
                pqp  += q_l[0,:,:,j]*q_l[1,:,:,j]*q_l[0,:,:,a]

        ppqq += q_l1[0,:,:,i]**2 + q_l1[1,:,:,i]**2

    den = np.sqrt(1/beta**4 + 1/beta**2 * ppqq + sum2)

    p_a = ((- r*(q_l[0,:,:,a] - du[0,:,:,a]) + mu_k[0,:,:,a])*den + 1/2 * qpq)/(2 * (1/beta**2 + 1/2 * qq))
    q_a = ((- r*(q_l[1,:,:,a] - du[1,:,:,a]) + mu_k[1,:,:,a])*den + 1/2 * pqp)/(2 * (1/beta**2 + 1/2 * pp))

    return p_a, q_a

In [ ]:
def update_qnew(x_initial, du, mu_k, beta, r, num_iterations = 5, epsilon = 1e-9):
    x    = x_initial
    x_t  = x
    x_tt = x

    for i in range(num_iterations):
        p0, q0 = q_a(x, x_t, x_tt, du, mu_k, beta, r, 0)
        p1, q1 = q_a(x, x_t, x_tt, du, mu_k, beta, r, 1)
        p2, q2 = q_a(x, x_t, x_tt, du, mu_k, beta, r, 2)

        x_tt = x_t
        x_t  = x
        x[0] = np.dstack((p0, p1, p2))
        x[1] = np.dstack((q0, q1, q2))
    return x

In [ ]:
def update_lambda(lambda_k, q_k, v_k, r):
    return lambda_k + r*(q_k - calcular_gradiente1D(v_k))

In [ ]:
def AL_ROF(f, K, K_ast, alpha, r, max_iteration = 20, gamma = 1.2, epsilon = 1e-12):
    #f        = normalize(f)
    u_k      = f
    q_k      = calcular_gradiente1D(f)
    lambda_k = np.zeros_like(q_k)

    for k in range(max_iteration):
        u_k2 = u_k
        u_k  = update_u(f, q_k, lambda_k, alpha, K, K_ast, r)

        if norm(u_k - u_k2)/norm(u_k2) < epsilon:
            break

        q_k = update_q(u_k, lambda_k, r)
        #q_k = np.array(np.gradient(u_k))

        lambda_k = update_lambda(lambda_k, q_k, u_k, r)
        r = r*gamma

    return u_k

## Tests

In [ ]:
img = cv2.imread(r"C:\Users\merce\Downloads\mariposa.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
cv2.imwrite(r'C:\Users\merce\Downloads\mariposa_gris.jpg', img)

True

In [ ]:
cv2.imshow('Imagen', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
f_denoise = img + np.random.normal(1, 25, img.shape)

cv2.imshow('Imagen', normalize(f_denoise))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.imwrite(r'C:\Users\merce\Downloads\mariposa_gris_ruido.jpg', f_denoise)

True

In [ ]:
I = np.array([[0,0,0], [0,1,0], [0,0,0]])

In [ ]:
import time

# Inicio de la medición del tiempo
inicio = time.perf_counter()

# Código que deseas medir
u = AL_ROF(f_denoise, I, I, 1, .05, 20, 1)

# Fin de la medición del tiempo
fin = time.perf_counter()

# Cálculo de la duración
duracion = fin - inicio

print(f"El código tardó {duracion} segundos en ejecutarse.")

El código tardó 13.57441340002697 segundos en ejecutarse.


In [ ]:
from skimage.metrics import peak_signal_noise_ratio
psnr_value = peak_signal_noise_ratio(normalize(img), normalize(u))

print(f"The PSNR value is: {psnr_value:.2f}")

The PSNR value is: 16.03


In [ ]:
cv2.imwrite(r'C:\Users\merce\Downloads\mariposa_05_20.jpg', u)

True

In [ ]:
cv2.imshow('Resultado', normalize(u))
cv2.imshow('Imagen con ruido',  normalize(f_denoise))
cv2.imshow('Imagen original',  img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# ***Three channel images***

In [ ]:
def update_u2(f, q_k, lambda_k, alpha, K, K_ast, r):
    n, m = f.shape
    #, Dx, Dy = get_kernels(n,m)

    mu_q0 = lambda_k[0] + r*q_k[0]
    mu_q1 = lambda_k[1] + r*q_k[1]

    L = fft2(np.array([[0, 1, 0],[1, -4, 1],[0, 1, 0]]),s = (n,m))
    Dx = fft2(np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]]),s = (n,m))
    Dy = fft2(np.array( [[-1, -1, -1], [0, 0, 0], [1, 1, 1]]),s = (n,m))

    print(L.shape, mu_q0.shape)

    num = alpha*fft2(K_ast,s = (n,m))*fft2(f) - Dx*mu_q0 - Dy*mu_q1
    den = alpha*fft2(K_ast,s = (n,m))*fft2(K,s = (n,m)) - r*L

    u = np.abs(ifft2(num / den))

    return u

In [ ]:
def calcular_gradiente(imagen):
    """
    Calcula el gradiente de una imagen a color utilizando el método de las diferencias finitas.

    Parámetros:
        imagen (numpy.ndarray): La imagen de entrada en formato RGB. Debe ser una matriz de tamaño (alto, ancho, 3).

    Retorna:
        gradiente (numpy.ndarray): El gradiente de la imagen, con las derivadas parciales en cada canal de color.
                                   Tiene el mismo tamaño que la imagen de entrada.
    """
    alto, ancho, _ = imagen.shape

    # Inicializar matrices para las derivadas parciales en cada canal de color
    dI_dx = np.zeros_like(imagen)
    dI_dy = np.zeros_like(imagen)

    # Calcular las derivadas parciales utilizando diferencias finitas
    for c in range(3):  # Iterar sobre los canales de color (R, G, B)
        dI_dx[:, 1:ancho, c] = imagen[:, 1:ancho, c] - imagen[:, :ancho-1, c]  # Diferencia en dirección x
        dI_dy[1:alto, :, c] = imagen[1:alto, :, c] - imagen[:alto-1, :, c]  # Diferencia en dirección y


    # Calcular el gradiente total combinando los gradientes de cada canal
    gradient = np.array([dI_dx, dI_dy])

    return gradient

In [ ]:
def AL_ROF_COLOR(f, K, K_ast, alpha, r, max_iteration = 20, gamma = 1.2, epsilon = 1e-30):
    g        = f.copy()
    u_k      = g
    q_k      = calcular_gradiente(f)
    lambda_k = np.zeros_like(q_k)

    print(q_k.shape)
    for k in range(max_iteration):
        u_k2 = u_k

        for i in range(3):
            u_k[:,:,i] = update_u2(g[:,:,i], q_k[:,:,:,i], lambda_k[:,:,:,i], alpha, K, K_ast, r)

        if norm(u_k - u_k2)/norm(u_k2) < epsilon:
            break

        for i in range(3):
            q_k[:,:,:,i] = update_q(u_k[:,:,i], lambda_k[:,:,:,i], r)
            #q_k[:,:,:,i] = np.array(np.gradient(u_k))

        lambda_k = lambda_k + r*(q_k - calcular_gradiente(u_k))
        r = r*gamma

    return u_k

## Tests

In [ ]:
img = cv2.imread(r"C:\Users\merce\Downloads\mariposa.jpg")
#img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
cv2.imshow('Imagen', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
f_denoise = img + np.random.normal(1, 25, img.shape)

cv2.imshow('Imagen', normalize(f_denoise))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.imwrite(r'C:\Users\merce\Downloads\astro_denoise.jpg', f_denoise)

True

In [ ]:
# Inicio de la medición del tiempo
inicio = time.perf_counter()

# Código que deseas medir
u = AL_ROF_COLOR(f_denoise, I, I, .05, .05, 10, 1)

# Fin de la medición del tiempo
fin = time.perf_counter()

# Cálculo de la duración
duracion = fin - inicio

print(f"El código tardó {duracion} segundos en ejecutarse.")

(2, 802, 1200, 3)
(802, 1200) (802, 1200)
(802, 1200) (802, 1200)
(802, 1200) (802, 1200)
El código tardó 2.7292250000173226 segundos en ejecutarse.


In [ ]:
cv2.imshow('Resultado', normalize(u))
cv2.imshow('Imagen original',  img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
psnr_value = peak_signal_noise_ratio(normalize(img), normalize(u))

print(f"The PSNR value is: {psnr_value:.2f}")

The PSNR value is: 23.36


In [ ]:
cv2.imwrite(r'C:\Users\merce\Downloads\astro_05.jpg', u)

True

In [ ]:
f_deblur = cv2.GaussianBlur(img, (15, 15), 0)

In [ ]:
cv2.imwrite(r'C:\Users\merce\Downloads\blured_mariposa.jpg', f_deblur)

True

In [ ]:
cv2.imshow('Imagen', normalize(f_deblur))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
vec = np.array([[1, 4, 6, 4, 1]])
K = (1/256)*vec.T@vec
K_ast = K.T #np.conjugate(K.T)

In [ ]:
# Inicio de la medición del tiempo
inicio = time.perf_counter()

# Código que deseas medir
u = AL_ROF_COLOR(normalize(f_deblur), K, K_ast, 8, 10, 4, 1.1)

# Fin de la medición del tiempo
fin = time.perf_counter()

# Cálculo de la duración
duracion = fin - inicio

print(f"El código tardó {duracion} segundos en ejecutarse.")

(2, 700, 960, 3)
(700, 960) (700, 960)
(700, 960) (700, 960)
(700, 960) (700, 960)
El código tardó 1.2147386000142433 segundos en ejecutarse.


In [ ]:
cv2.imshow('Resultado', normalize(u))
cv2.imshow('Imagen original',  img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
from PIL import Image

pil_image = Image.fromarray(u.astype(np.uint8))

# Save the image to a file
pil_image.save(r'C:\Users\merce\Downloads\generated_image.jpg')

In [ ]:
cv2.imwrite(r'C:\Users\merce\Downloads\deblured_mariposa.jpg', normalize(u).astype(np.uint8))

True